In [1]:
#Import Dependensies
import pandas as pd
import matplotlib.pyplot as plt
import csv
from datetime import datetime

import sqlalchemy
from sqlalchemy.exc import IntegrityError
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session, relationship, backref
from sqlalchemy import create_engine

from sqlalchemy.ext.declarative import declarative_base # extract classes into tables
from sqlalchemy import (Column, Integer, String, Float, Date, Table, MetaData, 
                        select, ForeignKey) # allow us to declare the column 

In [2]:
# Read CSV
csv_path ='./Resources/clean_hawaii_measurements.csv'
measurements = pd.read_csv(csv_path, encoding='ISO-8859-1')
measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [3]:
# Read CSV
csv_path ='./Resources/hawaii_stations.csv'
stations_data = pd.read_csv(csv_path, encoding='ISO-8859-1')
stations_data.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7.0
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [4]:
Base = declarative_base() 

In [5]:
class Measurement(Base):
    __tablename__= "measurement"
    #station = Column(String(255), ForeignKey('station.station'), primary_key=True)
    station_id = Column(String(255), ForeignKey('station.station'), primary_key=True)
    date = Column(Date, primary_key=True)
    prcp = Column(Float)
    tobs = Column(Integer)
    #stations = relationship("Station", backref=backref('measurement', order_by=date))
    
class Station(Base):
    __tablename__= "station"
    station = Column(String(255), primary_key=True)
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [6]:
engine = create_engine('sqlite:///hawaii.sqlite')

In [7]:
Base.metadata.create_all(engine)

In [8]:
Base.metadata.tables

immutabledict({'measurement': Table('measurement', MetaData(bind=None), Column('station_id', String(length=255), ForeignKey('station.station'), table=<measurement>, primary_key=True, nullable=False), Column('date', Date(), table=<measurement>, primary_key=True, nullable=False), Column('prcp', Float(), table=<measurement>), Column('tobs', Integer(), table=<measurement>), schema=None), 'station': Table('station', MetaData(bind=None), Column('station', String(length=255), table=<station>, primary_key=True, nullable=False), Column('name', String(length=255), table=<station>), Column('latitude', Float(), table=<station>), Column('longitude', Float(), table=<station>), Column('elevation', Float(), table=<station>), schema=None)})

In [9]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [10]:
def csvtoDictList(csvFilePath):
    dictlist=[]
    csvfile = open(csvFilePath)
    csvReader = csv.reader(csvfile)
    keys = csvReader.__next__()
    keys[0] = str(keys[0] + '_id')
    for line in csvReader:
        tmp = line[1].split('-')
        line[1] = datetime(int(tmp[0]),int(tmp[1]),int(tmp[2]))
        line[2] = float(line[2])
        line[3] = int(line[3])
        dictlist.append(dict(zip(keys,line)))
    return dictlist

def csv2DictList(csvFilePath):
    dictlist=[]
    csvfile = open(csvFilePath)
    csvReader = csv.reader(csvfile)
    keys = csvReader.__next__()
    for line in csvReader:
        line[2] = float(line[2])
        line[3] = float(line[3])
        line[4] = float(line[4])
        dictlist.append(dict(zip(keys,line)))
    return dictlist

In [11]:
measurement_data = csvtoDictList('./Resources/clean_hawaii_measurements.csv')
station_data = csv2DictList('./Resources/hawaii_stations.csv')
meta = MetaData()
meta.reflect(bind=engine)
measurement_table=meta.tables['measurement']
station_table=meta.tables['station']

In [12]:
conn = engine.connect() 
conn.execute(station_table.insert(), station_data)
conn.execute(measurement_table.insert(), measurement_data)

In [13]:
conn = engine.connect()
stmt=measurement_table.select().limit(5)
stmt2=station_table.select().limit(5)
rs=conn.execute(stmt).fetchall()
rs2=conn.execute(stmt2).fetchall()
conn.close()

In [14]:
for idx, row in enumerate(rs2):
    print((rs2[idx]).station, (rs2[idx]).name, (rs2[idx]).longitude, 
          (rs2[idx]).latitude, (rs2[idx]).elevation)

USC00519397 WAIKIKI 717.2, HI US -157.8168 21.2716 3.0
USC00513117 KANEOHE 838.1, HI US -157.8015 21.4234 14.6
USC00514830 KUALOA RANCH HEADQUARTERS 886.9, HI US -157.8374 21.5213 7.0
USC00517948 PEARL CITY, HI US -157.9751 21.3934 11.9
USC00518838 UPPER WAHIAWA 874.3, HI US -158.0111 21.4992 306.6


In [15]:
for idx, row in enumerate(rs):
    print((rs[idx]).station_id, (rs[idx]).date, (rs[idx]).prcp, (rs[idx]).tobs)

USC00519397 2010-01-01 0.08 65
USC00519397 2010-01-02 0.0 63
USC00519397 2010-01-03 0.0 74
USC00519397 2010-01-04 0.0 76
USC00519397 2010-01-07 0.06 70
